In [1]:
pip install selenium webdriver-manager pandas


Defaulting to user installation because normal site-packages is not writeable
  Using cached PySocks-1.7.1-py3-none-any.whl.metadata (13 kB)
   ---------------------------------------- 0.0/9.6 MB ? eta -:--:--
   -- ------------------------------------- 0.5/9.6 MB 4.2 MB/s eta 0:00:03
   -------- ------------------------------- 2.1/9.6 MB 6.9 MB/s eta 0:00:02
   -------------- ------------------------- 3.4/9.6 MB 6.9 MB/s eta 0:00:01
   --------------------- ------------------ 5.2/9.6 MB 7.6 MB/s eta 0:00:01
   ----------------------------- ---------- 7.1/9.6 MB 7.6 MB/s eta 0:00:01
   ------------------------------------ --- 8.7/9.6 MB 7.9 MB/s eta 0:00:01
   ---------------------------------------- 9.6/9.6 MB 7.4 MB/s  0:00:01
Using cached PySocks-1.7.1-py3-none-any.whl (16 kB)

   ------- --------------------------------  2/11 [wsproto]
   ------------------ ---------------------  5/11 [async-generator]
   --------------------- ------------------  6/11 [webdriver-manager]
   -------


[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time
import re

In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time

# Setup
options = webdriver.ChromeOptions()
options.add_argument('--disable-blink-features=AutomationControlled')
options.add_argument('--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36')

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

try:
    url = "https://cellphones.com.vn/mobile.html"
    driver.get(url)
    print("✓ Đã truy cập trang")
    time.sleep(5)
    
    # Đếm sản phẩm ban đầu
    products_initial = driver.find_elements(By.CSS_SELECTOR, '.product-info')
    print(f"✓ Số sản phẩm ban đầu: {len(products_initial)}")
    
    # Click button
    click_count = 0
    max_clicks = 100
    
    while click_count < max_clicks:
        try:
            # Scroll CHẬM xuống để button xuất hiện
            current_scroll = driver.execute_script("return window.pageYOffset;")
            driver.execute_script(f"window.scrollTo(0, {current_scroll + 800});")
            time.sleep(2)
            
            # Tìm TẤT CẢ button (bao gồm cả a tag, div...)
            all_buttons = driver.find_elements(By.XPATH, "//*[contains(text(), 'Xem thêm')]")
            
            print(f"\n--- Tìm thấy {len(all_buttons)} elements có 'Xem thêm' ---")
            
            # In ra TẤT CẢ
            product_button = None
            for idx, btn in enumerate(all_buttons):
                try:
                    btn_text = btn.text
                    btn_tag = btn.tag_name
                    btn_class = btn.get_attribute('class')
                    print(f"{idx+1}. Tag: {btn_tag}, Class: {btn_class}")
                    print(f"   Text: '{btn_text}'")
                    
                    # Tìm button có "sản phẩm" KHÔNG có "bình luận"
                    if 'sản phẩm' in btn_text.lower() and 'bình luận' not in btn_text.lower():
                        product_button = btn
                        print(f"   ✓✓✓ CHỌN BUTTON NÀY! ✓✓✓")
                except:
                    continue
            
            if not product_button:
                print("\n✗ Không tìm thấy button 'Xem thêm sản phẩm' lần này")
                print("  → Thử scroll thêm...")
                
                # Nếu đã scroll quá 3 lần mà không thấy thì dừng
                if click_count == 0:
                    # Chưa click lần nào, tiếp tục scroll tìm
                    time.sleep(2)
                    continue
                else:
                    # Đã click rồi mà không thấy nữa = hết
                    break
            
            # Click button
            print(f"\n✓ CLICK LẦN {click_count + 1}")
            driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", product_button)
            time.sleep(1)
            driver.execute_script("arguments[0].click();", product_button)
            click_count += 1
            
            # Đợi load
            time.sleep(4)
            
            # Đếm sản phẩm
            products_current = driver.find_elements(By.CSS_SELECTOR, '.product-info')
            print(f"  → Tổng sản phẩm hiện tại: {len(products_current)}")
            
        except Exception as e:
            print(f"\n✓ Lỗi hoặc đã hết")
            print(f"  Chi tiết: {str(e)[:150]}")
            break
    
    # Kết quả cuối
    time.sleep(2)
    products_final = driver.find_elements(By.CSS_SELECTOR, '.product-info')
    print(f"\n{'='*50}")
    print(f"✓ TỔNG CLICK: {click_count}")
    print(f"✓ TỔNG SẢN PHẨM: {len(products_final)}")
    print(f"{'='*50}")
    
    driver.save_screenshot('step2_result.png')
    print("✓ Screenshot: step2_result.png")
    
    input("\nEnter để đóng...")
    
finally:
    driver.quit()


✓ Đã truy cập trang
✓ Số sản phẩm ban đầu: 35

--- Tìm thấy 4 elements có 'Xem thêm' ---
1. Tag: a, Class: button btn-show-more button__show-more-product
   Text: 'Xem thêm 910 sản phẩm'
   ✓✓✓ CHỌN BUTTON NÀY! ✓✓✓
2. Tag: a, Class: btn-show-more button__content-show-more
   Text: 'Xem thêm'
3. Tag: button, Class: btn-show-more button__cmt-showmore
   Text: 'Xem thêm 1555 bình luận'
4. Tag: script, Class: 
   Text: ''

✓ CLICK LẦN 1
  → Tổng sản phẩm hiện tại: 55

--- Tìm thấy 4 elements có 'Xem thêm' ---
1. Tag: a, Class: button btn-show-more button__show-more-product
   Text: 'Xem thêm 888 sản phẩm'
   ✓✓✓ CHỌN BUTTON NÀY! ✓✓✓
2. Tag: a, Class: btn-show-more button__content-show-more
   Text: 'Xem thêm'
3. Tag: button, Class: btn-show-more button__cmt-showmore
   Text: 'Xem thêm 1555 bình luận'
4. Tag: script, Class: 
   Text: ''

✓ CLICK LẦN 2
  → Tổng sản phẩm hiện tại: 75

--- Tìm thấy 4 elements có 'Xem thêm' ---
1. Tag: a, Class: button btn-show-more button__show-more-product
  

KeyboardInterrupt: 

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time
import re

# Setup Chrome
options = webdriver.ChromeOptions()
options.add_argument('--disable-blink-features=AutomationControlled')
options.add_argument('--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36')

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

laptops_data = []

try:
    url = "https://www.thegioididong.com/laptop"
    driver.get(url)
    print("✓ Đã truy cập trang")
    time.sleep(5)
    
    # ===== BƯỚC 1: MỞ HẾT BUTTON "XEM THÊM" =====
    print("\n" + "="*60)
    print("BƯỚC 1: MỞ HẾT SẢN PHẨM")
    print("="*60)
    
    click_count = 0
    max_clicks = 100
    consecutive_fails = 0
    
    while click_count < max_clicks:
        try:
            # Scroll xuống
            current_scroll = driver.execute_script("return window.pageYOffset;")
            driver.execute_script(f"window.scrollTo(0, {current_scroll + 800});")
            time.sleep(2)
            
            # Tìm button "Xem thêm"
            try:
                view_more_button = driver.find_element(By.CSS_SELECTOR, '.view-more a')
                
                # Kiểm tra xem button có hiển thị không
                if view_more_button.is_displayed():
                    # Lấy số sản phẩm còn lại
                    try:
                        remain_element = driver.find_element(By.CSS_SELECTOR, '.view-more .remain')
                        remain_text = remain_element.text.strip()
                        print(f"  ✓ Click lần {click_count + 1} | Còn lại: {remain_text} laptop")
                    except:
                        print(f"  ✓ Click lần {click_count + 1}")
                    
                    # Click
                    driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", view_more_button)
                    time.sleep(1)
                    driver.execute_script("arguments[0].click();", view_more_button)
                    click_count += 1
                    time.sleep(4)
                    
                    # Đếm sản phẩm hiện tại
                    products_now = driver.find_elements(By.CSS_SELECTOR, 'li.item')
                    print(f"    → Tổng sản phẩm: {len(products_now)}")
                    
                    consecutive_fails = 0
                else:
                    consecutive_fails += 1
                    
            except:
                consecutive_fails += 1
            
            if consecutive_fails >= 3:
                print(f"✓ Đã load hết sau {click_count} lần click")
                break
                
        except Exception as e:
            consecutive_fails += 1
            if consecutive_fails >= 3:
                print(f"✓ Dừng sau {click_count} lần click")
                break
            time.sleep(2)
    
    print(f"\n✓ Đã click {click_count} lần")
    
    # ===== BƯỚC 2: LẤY TẤT CẢ LINKS SẢN PHẨM =====
    print("\n" + "="*60)
    print("BƯỚC 2: LẤY LINKS SẢN PHẨM")
    print("="*60)
    
    time.sleep(3)
    products = driver.find_elements(By.CSS_SELECTOR, 'li.item a')
    print(f"✓ Tổng sản phẩm trên trang: {len(products)}")
    
    product_links = []
    for product in products:
        try:
            link = product.get_attribute('href')
            if link and '/laptop/' in link and link not in product_links:
                product_links.append(link)
        except:
            continue
    
    print(f"✓ Đã lấy {len(product_links)} links unique\n")
    
    # ===== BƯỚC 3: CRAWL CHI TIẾT TỪNG SẢN PHẨM =====
    print("="*60)
    print(f"BƯỚC 3: CRAWL {len(product_links)} SẢN PHẨM")
    print("="*60 + "\n")
    
    for idx, link in enumerate(product_links, 1):
        try:
            print(f"[{idx}/{len(product_links)}] Crawling...")
            driver.get(link)
            time.sleep(3)
            
            laptop_info = {
                'name': '',
                'cpu': '',
                'ram': '',
                'gpu': '',
                'storage': '',
                'screen_size': '',
                'screen_resolution': '',
                'screen_panel': '',
                'battery': '',
                'color': '',
                'price': '',
                'rating': '',
                'review_text': ''
            }
            
            # === NAME ===
            try:
                name_element = driver.find_element(By.CSS_SELECTOR, '.product-name h1')
                laptop_info['name'] = name_element.text.strip()
                print(f"  ✓ Name: {laptop_info['name'][:40]}")
            except:
                pass
            
            # === ĐỌC SPECS TỪ BOX ACTIVE ===
            try:
                active_spec_lists = driver.find_elements(By.CSS_SELECTOR, 'ul.text-specifi.active')
                
                for spec_list in active_spec_lists:
                    items = spec_list.find_elements(By.TAG_NAME, 'li')
                    
                    for item in items:
                        try:
                            asides = item.find_elements(By.TAG_NAME, 'aside')
                            if len(asides) >= 2:
                                label = asides[0].text.lower().strip()
                                value = asides[1].text.strip()
                                
                                if 'công nghệ cpu' in label and not laptop_info['cpu']:
                                    laptop_info['cpu'] = value
                                    print(f"  ✓ CPU: {value}")
                                elif 'ram:' in label and not laptop_info['ram']:
                                    laptop_info['ram'] = value.split('\n')[0] if '\n' in value else value
                                    print(f"  ✓ RAM: {laptop_info['ram']}")
                                elif 'card màn hình' in label and not laptop_info['gpu']:
                                    laptop_info['gpu'] = value
                                    print(f"  ✓ GPU: {value[:40]}")
                                elif 'ổ cứng' in label and not laptop_info['storage']:
                                    laptop_info['storage'] = value.split('\n')[0] if '\n' in value else value
                                    print(f"  ✓ Storage: {laptop_info['storage']}")
                                elif 'kích thước màn hình' in label and not laptop_info['screen_size']:
                                    laptop_info['screen_size'] = value
                                    print(f"  ✓ Screen Size: {value}")
                                elif 'độ phân giải' in label and not laptop_info['screen_resolution']:
                                    laptop_info['screen_resolution'] = value
                                    print(f"  ✓ Screen Resolution: {value}")
                                elif 'tấm nền' in label and not laptop_info['screen_panel']:
                                    laptop_info['screen_panel'] = value
                                    print(f"  ✓ Screen Panel: {value}")
                                elif 'thông tin pin' in label and not laptop_info['battery']:
                                    laptop_info['battery'] = value
                                    print(f"  ✓ Battery: {value}")
                        except:
                            continue
            except:
                pass
            
            # === MỞ CÁC BOX KHÁC ===
            try:
                spec_headers = driver.find_elements(By.CSS_SELECTOR, '.box-specifi > a:not(.active)')
                for header in spec_headers:
                    try:
                        driver.execute_script("arguments[0].click();", header)
                        time.sleep(0.3)
                    except:
                        continue
                
                time.sleep(1)
                all_spec_lists = driver.find_elements(By.CSS_SELECTOR, 'ul.text-specifi')
                
                for spec_list in all_spec_lists:
                    items = spec_list.find_elements(By.TAG_NAME, 'li')
                    
                    for item in items:
                        try:
                            asides = item.find_elements(By.TAG_NAME, 'aside')
                            if len(asides) >= 2:
                                label = asides[0].text.lower().strip()
                                value = asides[1].text.strip()
                                
                                if 'ram:' in label and not laptop_info['ram']:
                                    laptop_info['ram'] = value.split('\n')[0] if '\n' in value else value
                                    print(f"  ✓ RAM: {laptop_info['ram']}")
                                elif 'card màn hình' in label and not laptop_info['gpu']:
                                    laptop_info['gpu'] = value
                                    print(f"  ✓ GPU: {value[:40]}")
                                elif 'ổ cứng' in label and not laptop_info['storage']:
                                    laptop_info['storage'] = value.split('\n')[0] if '\n' in value else value
                                    print(f"  ✓ Storage: {laptop_info['storage']}")
                                elif 'kích thước màn hình' in label and not laptop_info['screen_size']:
                                    laptop_info['screen_size'] = value
                                    print(f"  ✓ Screen Size: {value}")
                                elif 'độ phân giải' in label and not laptop_info['screen_resolution']:
                                    laptop_info['screen_resolution'] = value
                                    print(f"  ✓ Screen Resolution: {value}")
                                elif 'tấm nền' in label and not laptop_info['screen_panel']:
                                    laptop_info['screen_panel'] = value
                                    print(f"  ✓ Screen Panel: {value}")
                                elif 'thông tin pin' in label and not laptop_info['battery']:
                                    laptop_info['battery'] = value
                                    print(f"  ✓ Battery: {value}")
                        except:
                            continue
            except:
                pass
            
            # === COLOR ===
            try:
                active_color = driver.find_element(By.CSS_SELECTOR, '.box03.color .box03__item.act')
                color_text = active_color.text.strip()
                laptop_info['color'] = color_text.split()[-1] if color_text.split() else color_text
                print(f"  ✓ Color: {laptop_info['color']}")
            except:
                pass
            
            # === PRICE ===
            try:
                price_element = driver.find_element(By.CSS_SELECTOR, '.bs_price strong')
                laptop_info['price'] = price_element.text.strip()
                print(f"  ✓ Price (Sale): {laptop_info['price']}")
            except:
                try:
                    price_element = driver.find_element(By.CSS_SELECTOR, '.warranty-options .option.active .option-price')
                    laptop_info['price'] = price_element.text.strip().split('\n')[0].strip()
                    print(f"  ✓ Price: {laptop_info['price']}")
                except:
                    try:
                        price_element = driver.find_element(By.CSS_SELECTOR, '.box-price-present')
                        laptop_info['price'] = price_element.text.strip()
                        print(f"  ✓ Price: {laptop_info['price']}")
                    except:
                        pass
            
            # === RATING ===
            try:
                rating_score = driver.find_element(By.CSS_SELECTOR, '.point-average-score')
                rating_total = driver.find_element(By.CSS_SELECTOR, '.point-average-total')
                laptop_info['rating'] = f"{rating_score.text.strip()}{rating_total.text.strip()}"
                print(f"  ✓ Rating: {laptop_info['rating']}")
            except:
                pass
            
            # === REVIEW TEXT ===
            try:
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                time.sleep(2)
                
                review_texts = []
                review_items = driver.find_elements(By.CSS_SELECTOR, 'ul.comment-list li.par')
                
                if review_items:
                    for review_item in review_items[:5]:
                        try:
                            comment_element = review_item.find_element(By.CSS_SELECTOR, '.cmt-txt')
                            comment_text = comment_element.text.strip()
                            
                            if comment_text and len(comment_text) > 10:
                                comment_text = comment_text.replace('\n', ' ').replace('\r', ' ')
                                comment_text = re.sub(r'\s+', ' ', comment_text).strip()
                                review_texts.append(comment_text)
                        except:
                            continue
                    
                    if review_texts:
                        laptop_info['review_text'] = ' || '.join(review_texts)
                        print(f"  ✓ Reviews: {len(review_texts)} reviews")
            except:
                pass
            
            laptops_data.append(laptop_info)
            print(f"  ✓ Hoàn thành\n")
            
            # === CHECKPOINT MỖI 50 SẢN PHẨM ===
            if idx % 50 == 0:
                df_checkpoint = pd.DataFrame(laptops_data)
                column_order = ['name', 'cpu', 'ram', 'gpu', 'storage', 'screen_size', 'screen_resolution', 
                               'screen_panel', 'battery', 'color', 'price', 'rating', 'review_text']
                df_checkpoint = df_checkpoint[column_order]
                df_checkpoint.to_csv(f'checkpoint_tgdd_{idx}.csv', index=False, encoding='utf-8-sig', quoting=1, escapechar='\\')
                print(f"\n{'='*60}")
                print(f"💾 CHECKPOINT: {idx} SẢN PHẨM ĐÃ LƯU")
                print(f"File: checkpoint_tgdd_{idx}.csv")
                print(f"{'='*60}\n")
            
        except Exception as e:
            print(f"  ✗ LỖI: {str(e)[:80]}\n")
            continue
    
    # ===== LƯU KẾT QUẢ CUỐI CÙNG =====
    df = pd.DataFrame(laptops_data)
    column_order = ['name', 'cpu', 'ram', 'gpu', 'storage', 'screen_size', 'screen_resolution', 
                   'screen_panel', 'battery', 'color', 'price', 'rating', 'review_text']
    df = df[column_order]
    
    df.to_csv('thegioididong_laptop_full.csv', index=False, encoding='utf-8-sig', quoting=1, escapechar='\\')
    
    print(f"\n{'='*60}")
    print(f"✓ HOÀN THÀNH!")
    print(f"{'='*60}")
    print(f"Số lần click 'Xem thêm': {click_count}")
    print(f"Tổng sản phẩm crawl: {len(laptops_data)}")
    print(f"Có rating: {sum(1 for p in laptops_data if p['rating'])}")
    print(f"Có review: {sum(1 for p in laptops_data if p['review_text'])}")
    print(f"\nFile output: thegioididong_laptop_full.csv")
    print(f"Checkpoint files: checkpoint_tgdd_50.csv, checkpoint_tgdd_100.csv, ...")
    print(f"{'='*60}")
    
    input("\nNhấn Enter để đóng browser...")
    
finally:
    driver.quit()
